# Classes

## Resumo

Classes proporcionam uma forma de organizar dados e funcionalidades juntos. Criar uma nova classe cria um novo `tipo` de objeto, permitindo que novas `instâncias` desse tipo sejam produzidas. Cada instância da classe pode ter atributos anexados a ela, para manter seu estado. Instâncias da classe também podem ter métodos (definidos pela classe) para modificar seu estado.

Em comparação com outras linguagens de programação, o mecanismo de classes de Python introduz a programação orientada a objetos sem acrescentar muitas novidades de sintaxe ou semântica. É uma mistura de mecanismos equivalentes encontrados em C++ e Modula-3. As classes em Python oferecem todas as características tradicionais da programação orientada a objetos: o mecanismo de herança permite múltiplas classes base (herança múltipla), uma classe derivada pode sobrescrever quaisquer métodos de uma classe ancestral e um método pode invocar outro método homônimo de uma classe ancestral. Objetos podem armazenar uma quantidade arbitrária de dados de qualquer tipo. Assim como acontece com os módulos, as classes fazem parte da natureza dinâmica de Python: são criadas em tempo de execução, e podem ser alteradas após sua criação.

Usando a terminologia de C++, todos os membros de uma classe (incluindo dados) são públicos (veja exceção abaixo Variáveis privadas), e todos as funções membro são virtuais. Como em Modula-3, não existem atalhos para referenciar membros do objeto de dentro dos seus métodos: o método (função definida em uma classe) é declarado com um primeiro argumento explícito representando o objeto (instância da classe), que é fornecido implicitamente pela chamada ao método. Como em Smalltalk, classes são objetos. Isso fornece uma semântica para importar e renomear. Ao contrário de C++ ou Modula-3, tipos pré-definidos podem ser utilizados como classes base para extensões por herança pelo usuário. Também, como em C++, a maioria dos operadores (aritméticos, indexação, etc) podem ser redefinidos por instâncias de classe.

## 1. Uma palavra sobre nomes e objetos

Objetos têm individualidade, e vários nomes (em diferentes escopos) podem ser vinculados a um mesmo objeto. Isso é chamado de apelidamento em outras linguagens. Geralmente, esta característica não é muito apreciada, e pode ser ignorada com segurança ao lidar com tipos imutáveis (números, strings, tuplas). Entretanto, apelidamento pode ter um efeito surpreendente na semântica do código Python envolvendo objetos mutáveis como listas, dicionários e a maioria dos outros tipos. Isso pode ser usado em benefício do programa, porque os apelidos funcionam de certa forma como ponteiros. Por exemplo, passar um objeto como argumento é barato, pois só um ponteiro é passado na implementação; e se uma função modifica um objeto passado como argumento, o invocador verá a mudança — isso elimina a necessidade de ter dois mecanismos de passagem de parâmetros como em Pascal.

## 2. Escopos e espaços de nomes (NameSpaces) do Python

Antes de introduzir classes, é preciso falar das regras de escopo em Python. Definições de classe fazem alguns truques com espaços de nomes. Portanto, primeiro é preciso entender claramente como escopos e espaços de nomes funcionam, para entender o que está acontecendo. Esse conhecimento é muito útil para qualquer programador Python avançado.

Vamos começar com algumas definições.

Um espaço de nomes é um mapeamento que associa nomes a objetos. Atualmente, são implementados como dicionários em Python, mas isso não é perceptível (a não ser pelo desempenho), e pode mudar no futuro.<br>
Exemplos de espaços de nomes são :
- o conjunto de nomes pré-definidos (funções como [abs()](https://docs.python.org/pt-br/3/library/functions.html#abs) e as exceções pré-definidas);
- nomes globais em um módulo;
- e nomes locais na invocação de uma função.

De certa forma, os atributos de um objeto também formam um espaço de nomes. O mais importante é saber que não existe nenhuma relação entre nomes em espaços de nomes distintos. Por exemplo, dois módulos podem definir uma função de nome maximize sem confusão — usuários dos módulos devem prefixar a função com o nome do módulo, para evitar colisão.

A propósito, utilizo a palavra `atributo` para qualquer nome depois de um ponto. Na expressão `z.real`, por exemplo, `real` é um atributo do objeto `z`. Estritamente falando, referências para nomes em módulos são atributos: na expressão `modname.funcname`, `modname` é um objeto módulo e `funcname` é um de seus atributos. Neste caso, existe um mapeamento direto entre os atributos de um módulo e os nomes globais definidos no módulo: eles compartilham o mesmo espaço de nomes!

Atributos podem ser somente *leitura* ou para *leitura* e *escrita*. No segundo caso, é possível atribuir um novo valor ao atributo. Atributos de módulos são passíveis de atribuição: você pode escrever `modname.the_answer = 42`. Atributos que aceitam escrita também podem ser apagados através da instrução [del](https://docs.python.org/pt-br/3/reference/simple_stmts.html#del). Por exemplo, `del modname.the_answer` removerá o atributo `the_answer` do objeto referenciado por `modname`.

Espaços de nomes são criados em momentos diferentes e possuem diferentes ciclos de vida. O espaço de nomes que contém os nomes **built-in** é criado quando o interpretador inicializa e nunca é removido. O espaço de nomes global de um módulo é criado quando a definição do módulo é lida, e normalmente duram até a finalização do interpretador. Os comandos executados pela invocação do interpretador, pela leitura de um script como programa principal, ou interativamente, são parte do módulo chamado [__main__](https://docs.python.org/pt-br/3/library/__main__.html#module-__main__), e portanto possuem seu próprio espaço de nomes. (Os nomes **built-in** possuem seu próprio espaço de nomes no módulo chamado [builtins](https://docs.python.org/pt-br/3/library/builtins.html#module-builtins)).

O espaço de nomes local de uma função é criado quando a função é invocada, e apagado quando a função retorna ou levanta uma exceção que não é tratada na própria função. (Na verdade, uma forma melhor de descrever o que realmente acontece é que o espaço de nomes local é “esquecido” quando a função termina.) Naturalmente, cada invocação recursiva de uma função tem seu próprio espaço de nomes.

Um escopo é uma região textual de um programa Python onde um espaço de nomes é diretamente acessível. Aqui, *"diretamente acessível"* significa que uma referência sem um prefixo qualificador permite o acesso ao nome.

Ainda que escopos sejam determinados estaticamente, eles são usados dinamicamente. A qualquer momento durante a execução, existem 3 ou 4 escopos aninhados cujos espaços de nomes são diretamente acessíveis:
- o escopo mais interno, que é acessado primeiro, contem os nomes locais;
- os escopos das funções que envolvem a função atual, que são acessados a partir do escopo mais próximo, contêm nomes não-locais, mas também não-globais;
- o penúltimo escopo contém os nomes globais do módulo atual;
- e o escopo mais externo (acessado por último) contém os nomes das funções embutidas e demais objetos pré-definidos do interpretador;

Se um nome é declarado no escopo global, então todas as referências e atribuições de valores vão diretamente para o penúltimo escopo, que contém os nomes globais do módulo. Para alterar variáveis declaradas fora do escopo mais interno, a instrução [nonlocal](https://docs.python.org/pt-br/3/reference/simple_stmts.html#nonlocal) pode ser usada; caso contrário, todas essas variáveis serão apenas para leitura (a tentativa de atribuir valores a essas variáveis simplesmente criará uma nova variável local, no escopo interno, não alterando nada na variável de nome idêntico fora dele).

Normalmente, o escopo local referencia os nomes locais da função corrente no texto do programa. Fora de funções, o escopo local referencia os nomes do escopo global: espaço de nomes do módulo. Definições de classes adicionam um outro espaço de nomes ao escopo local.

É importante perceber que escopos são determinados estaticamente, pelo texto do código-fonte: o escopo global de uma função definida em um módulo é o espaço de nomes deste módulo, sem importar de onde ou por qual apelido a função é invocada. Por outro lado, a busca de nomes é dinâmica, ocorrendo durante a execução. Porém, a evolução da linguagem está caminhando para uma resolução de nomes estática, em “tempo de compilação”, portanto não conte com a resolução dinâmica de nomes! (De fato, variáveis locais já são resolvidas estaticamente).

Uma peculiaridade especial do Python é que – se nenhuma instrução [global](https://docs.python.org/pt-br/3/reference/simple_stmts.html#global) ou [nonlocal](https://docs.python.org/pt-br/3/reference/simple_stmts.html#nonlocal) estiver em vigor – as atribuições de nomes sempre entram no escopo mais interno. As atribuições não copiam dados — elas apenas vinculam nomes aos objetos. O mesmo vale para exclusões: a instrução `del x` remove a ligação de `x` do espaço de nomes referenciado pelo escopo local. De fato, todas as operações que introduzem novos nomes usam o escopo local: em particular, instruções [import](https://docs.python.org/pt-br/3/reference/simple_stmts.html#import) e definições de funções ligam o módulo ou o nome da função no escopo local.

A instrução [global](https://docs.python.org/pt-br/3/reference/simple_stmts.html#global) pode ser usada para indicar que certas variáveis residem no escopo global ao invés do local; a instrução [nonlocal](https://docs.python.org/pt-br/3/reference/simple_stmts.html#nonlocal) indica que variáveis particulares estão em um espoco mais interno e devem ser recuperadas lá.

### 2.1. Exemplo de escopos e espaço de nomes

Este é um exemplo que demonstra como se referir aos diferentes escopos e aos espaços de nomes, e como [global](https://docs.python.org/pt-br/3/reference/simple_stmts.html#global) e [nonlocal](https://docs.python.org/pt-br/3/reference/simple_stmts.html#nonlocal) pode afetar ligação entre as variáveis.

In [1]:
def teste_escopo():
    def cria_local():
        spam = "local spam"

    def cria_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def cria_global():
        global spam
        spam = "global spam"

    spam = "teste spam"
    cria_local()
    print("Depois da atribuição local :", spam)
    cria_nonlocal()
    print("Depois da atribuição nonlocal :", spam)
    cria_global()
    print("Depois da atribuição global :", spam)

teste_escopo()
print("No escopo global :", spam)

Depois da atribuição local : teste spam
Depois da atribuição nonlocal : nonlocal spam
Depois da atribuição global : nonlocal spam
No escopo global : global spam


A saída do código de exemplo é :

`Depois da atribuição local : teste spam`<br>
`Depois da atribuição nonlocal : nonlocal spam`<br>
`Depois da atribuição global : nonlocal spam`<br>
`No escopo global : global spam`

Observe como uma atribuição *local* (que é o padrão) não altera o vínculo de teste_escopo a spam. A instrução [nonlocal](https://docs.python.org/pt-br/3/reference/simple_stmts.html#nonlocal) mudou o vínculo de escopo_teste de spam e a atribuição [global](https://docs.python.org/pt-br/3/reference/simple_stmts.html#global) alterou a ligação para o nível do módulo.

Você também pode ver que não havia nenhuma ligação anterior para spam antes da atribuição [global](https://docs.python.org/pt-br/3/reference/simple_stmts.html#global).

## 3. Uma primeira olhada nas classes

Classes introduzem novidades sintáticas, três novos tipos de objetos, e também alguma semântica nova.

### 3.1. Sintaxe da definição de classe

## 1. Classe

São moldes para criar novos objetos. As classes geram novos objetos (instâncias) que podem ter seus próprios atributos e métodos.<br>
Os objetos gerados pela classe pode usar seus dados internos para realizar várias ações.<br>
Por convenção, usamos o **PascalCase** para nomes de classes.

In [ ]:
# nome é uma instância da classe str
nome = 'fulano'
print(nome.upper())

print(isinstance(nome, str))

### 1.1. Criando a classe e atributos

In [ ]:
class Pessoa:
    ...

pessoa1 = Pessoa()
print(pessoa1)

O que são atributos ?<br>
São propriedades que a minha classe vai possuir.

In [ ]:
class Pessoa:
    ...

p1 = Pessoa()
p1.nome = 'fulano'
p1.idade = 15

print(p1)
print(p1.nome)
print(p1.idade)

Mas e se quisermos criar mais uma classe? Não fica prático repetir.

In [ ]:
class Pessoa:
    ...

p1 = Pessoa()
p1.nome = 'fulano'
p1.idade = 15
p2 = Pessoa()
p2.nome = 'ciclano'
p2.idade = 22

print(p1)
print(p1.nome)
print(p1.idade)

print(p2)
print(p2.nome)
print(p2.idade)

### 1.2. \_\_init__ e self..

In [ ]:
class Pessoa():
    def __init__(self, nome, idade):
        self.nome = nome
        self.idade = idade

p1 = Pessoa('fulano', 15)
print(p1.nome)
print(p1.idade)

p2 = Pessoa('ciclano', 22)
print(p2.nome)
print(p2.idade)

### 1.3. Métodos

São os comportamentos da classe.

In [ ]:
class Carro():
    def __init__(self):
        # hard coded : quando tu escreve algo diretamente no código
        self.nome = 'Fusca'

fusca = Carro()
print(fusca.nome)

celta = Carro()
print(celta.nome)

Deixando a criação mais flexível.

In [ ]:
class Carro():
    def __init__(self, var='um carro'):
        # hard coded : quando tu escreve algo diretamente no código
        self.nome = var
    
    def buzinar(self):
        print(f'{self.nome} está buzinando!')

fusca = Carro('Fusca')
print(fusca.nome)
fusca.buzinar()

celta = Carro(var='Celta')
print(celta.nome)
celta.buzinar()

### 1.4. self

O **self** é uma convenção quando se quer referenciar a própria instância.<br>
Uma classe pode gerar várias instâncias. Na classe, o self é a própria instância.

In [ ]:
class Carro():
    def __init__(ola, var='um carro'):
        # hard coded : quando tu escreve algo diretamente no código
        ola.nome = var
    
    def buzinar(fomfom):
        print(f'{fomfom.nome} está buzinando!')

fusca = Carro('Fusca')
print(fusca.nome)
fusca.buzinar()

In [ ]:
class Carro():
    def __init__(self, var='um carro'):
        # hard coded : quando tu escreve algo diretamente no código
        self.nome = var
    
    def buzinar(self):
        print(f'{self.nome} está buzinando!')

fusca = Carro('Fusca')
fusca.buzinar()
Carro.buzinar(fusca)

### 1.5. Escopos das Classes

Assim como as funções, as classes também tem escopos global e local.

In [ ]:
class Carro():
    rodas = 4

print(Carro.rodas)
print(rodas)

Agora vai gerar erro, pois precisamos instanciar a classe para ter acesso aos seus atributos.

In [ ]:
class Carro():
    def __init__(self, rodas):
        self.rodas = rodas

print(Carro.rodas)

In [ ]:
class Carro():
    def __init__(self, rodas):
        self.rodas = rodas

        nome = 'fulano'
        print(nome)

    def buzina(self):
        print(f'O {nome} está buzinando!')

fusca = Carro(4)
print(fusca.rodas)
fusca.buzina()

Um método de classe também pode receber um valor de fora da classe.

In [ ]:
class Carro():
    def __init__(self, nome):
        self.nome = nome

        nome = 'fulano'
        print(nome)

    def buzina(self, lerdo='Lamborghini'):
        print(f'O {self.nome} está buzinando para a {lerdo}')

fusca = Carro('Fusca')
fusca.buzina('Ferrari')
fusca.buzina()

Usando o *args.

In [ ]:
class Carro():
    def __init__(self, nome):
        self.nome = nome

    def buzina(self, lerdo='Lamborghini'):
        return f'O {self.nome} está buzinando para a {lerdo}'
    
    def acao(self, *args):
        return self.buzina(*args)

fusca = Carro('Fusca')
print(fusca.acao('Ferrari'))
print(fusca.acao())

### 1.6. Estados Dentro da Classe

Mantendo estados dentro da classe.

In [ ]:
class Cachorro():
    def __init__(self, nome, latindo=False):
        self.nome = nome
        self.latindo = latindo
    
    def latir(self):
        if self.latindo:
            print(f'O {self.nome} ainda está latindo!')
            return
        
        print(f'O {self.nome} está latindo...')
        self.latindo = True
    
    def parar_latir(self):
        if not self.latindo:
            print(f'O {self.nome} não está latindo!')
            return
        
        print(f'O {self.nome} está parando de latir...')
        self.latindo = False
    
    def roncar(self):
        if self.latindo:
            print(f'O {self.nome} não pode roncar enquanto late!')
            return
        
        print(f'O {self.nome} está roncando...')

fido = Cachorro('Fido')
rex = Cachorro('Rex')

fido.latir()
fido.latir()
fido.roncar()
fido.parar_latir()
fido.roncar()

### 1.7. Atributos de Classe

São variáveis que são ligadas à classe.

In [ ]:
class Pessoa():
    ano_atual = 2023

    def __init__(self, nome, idade):
        self.nome = nome
        self.idade = idade

    def get_ano_nasc(self):
        return Pessoa.ano_atual - self.idade

p1 = Pessoa('João', 65)
p2 = Pessoa('Almeida', 25)
print(f'{Pessoa.ano_atual = }')

print(p1.get_ano_nasc())
print(p2.get_ano_nasc())
print(p1.__dict__)

### 1.8. \_\_dict__ e vars()

São maneiras de ver os atributos e valores de uma instância.

In [ ]:
class Pessoa():
    ano_atual = 2023

    def __init__(self, nome, idade):
        self.nome = nome
        self.idade = idade

    def get_ano_nasc(self):
        return Pessoa.ano_atual - self.idade

p1 = Pessoa('João', 65)
print(p1.__dict__)
print(vars(p1))

# ele pode ser editável, mas não é comum
p1.__dict__['novo'] = 'atributo'
print(p1.__dict__)
print(vars(p1))

# ou apagável
del p1.__dict__['idade']
print(p1.__dict__)
print(vars(p1))

Também é possível passar os dados de um dicionário para o objeto e criar (ou recriar) uma instância com eles.

In [ ]:
class Pessoa():
    ano_atual = 2023

    def __init__(self, nome, idade):
        self.nome = nome
        self.idade = idade

    def get_ano_nasc(self):
        return Pessoa.ano_atual - self.idade

dados = {'nome': 'João', 'idade': 65}
p1 = Pessoa(**dados)
print(p1.nome)
print(p1.idade)


João
65
